In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.max_columns=None
pd.options.display.max_rows=None

In [3]:
df = pd.read_csv('./especies_combu_polygon.csv')

In [4]:
df.head()

,key,datasetKey,publishingOrgKey,installationKey,hostingOrganizationKey,publishingCountry,protocol,lastCrawled,lastParsed,crawlId,extensions,basisOfRecord,occurrenceStatus,taxonKey,kingdomKey,phylumKey,classKey,orderKey,familyKey,genusKey,speciesKey,acceptedTaxonKey,scientificName,acceptedScientificName,kingdom,phylum,order,family,genus,species,genericName,specificEpithet,taxonRank,taxonomicStatus,iucnRedListCategory,dateIdentified,decimalLatitude,decimalLongitude,continent,stateProvince,gadm,year,month,day,eventDate,startDayOfYear,endDayOfYear,issues,modified,lastInterpreted,references,license,isSequenced,identifiers,media,facts,relations,isInCluster,datasetName,recordedBy,identifiedBy,geodeticDatum,class,countryCode,recordedByIDs,identifiedByIDs,gbifRegion,country,publishedByGbifRegion,rightsHolder,identifier,http://unknown.org/nick,verbatimEventDate,collectionCode,verbatimLocality,gbifID,occurrenceID,taxonID,catalogNumber,institutionCode,eventTime,http://unknown.org/captive,identificationID,lifeStage,coordinateUncertaintyInMeters,informationWithheld,occurrenceRemarks,infraspecificEpithet,individualCount,vernacularName,taxonConceptID,locality,sex,recordNumber,municipality,ownerInstitutionCode,collectionID,establishmentMeans,identificationRemarks,higherGeography,institutionKey,preparations,county,language,type,samplingProtocol,habitat,identificationVerificationStatus,eventType,networkKeys,collectionKey,otherCatalogNumbers,institutionID,fieldNotes,bibliographicCitation,elevation,elevationAccuracy,associatedSequences,dynamicProperties,associatedTaxa,rights,disposition,accessRights,nomenclaturalCode,verbatimElevation,georeferencedBy,http://unknown.org/recordID,georeferenceVerificationStatus,identificationQualifier,http://unknown.org/recordEnteredBy,higherClassification,georeferenceSources,typeStatus,georeferencedDate,georeferenceProtocol,locationAccordingTo,footprintWKT,verbatimCoordinateSystem,organismID,previousIdentifications
0,4854782149,50c9509d-22c7-4a22-a47d-8c48425ef4a7,28eb1a3f-1c15-4a95-931a-4af90ecb574d,997448a8-f762-11e1-a439-00145eb45e9a,28eb1a3f-1c15-4a95-931a-4af90ecb574d,US,DWC_ARCHIVE,2024-09-20T15:47:23.061+00:00,2024-09-21T09:37:16.371+00:00,486,{},HUMAN_OBSERVATION,PRESENT,3168943,6,7707728.0,220.0,414.0,8801.0,3168942.0,3168943.0,3168943.0,Hippobroma longiflora (L.) G.Don,Hippobroma longiflora (L.) G.Don,Plantae,Tracheophyta,Asterales,Campanulaceae,Hippobroma,Hippobroma longiflora,Hippobroma,longiflora,SPECIES,ACCEPTED,NE,2024-05-04T01:48:20,-1.504886,-48.462257,SOUTH_AMERICA,Pará,"{'level0': {'gid': 'BRA', 'name': 'Brazil'}, '...",2024.0,3.0,16.0,2024-03-16T12:54,76.0,76.0,"['COORDINATE_ROUNDED', 'CONTINENT_DERIVED_FROM...",2024-05-05T22:22:37.000+00:00,2024-09-21T09:37:16.371+00:00,https://www.inaturalist.org/observations/21330...,http://creativecommons.org/licenses/by-nc/4.0/...,False,[{'identifier': '213300731'}],[],[],[],False,iNaturalist research-grade observations,Fernando J. M. Rojas-Runjaic,Fernando J. M. Rojas-Runjaic,WGS84,Magnoliopsida,BR,[],[],LATIN_AMERICA,Brazil,NORTH_AMERICA,Fernando J. M. Rojas-Runjaic,213300731,rojas-runjaic,2024/03/16 12:54 PM,Observations,"Belém - Pará, Brasil",4854782149,https://www.inaturalist.org/observations/21330...,154827,213300731,iNaturalist,12:54:00-03:00,wild,481824413.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4868048393,50c9509d-22c7-4a22-a47d-8c48425ef4a7,28eb1a3f-1c15-4a95-931a-4af90ecb574d,997448a8-f762-11e1-a439-00145eb45e9a,28eb1a3f-1c15-4a95-931a-4af90ecb574d,US,DWC_ARCHIVE,2024-09-20T15:47:23.061+00:00,2024-09-21T09:37:19.948+00:00,486,{},HUMAN_OBSERVATION,PRESENT,3733807,6,7707728.0,220.0,408.0,4160847.0,8398900.0,3733807.0,3733807.0,Bonnaya antipoda (L.) Druce,Bonnaya antipoda (L.) Druce,Plantae,Tracheophyta,Lamiales,Linderniaceae,Bonnaya,Bonnaya antipoda,Bonnaya,an

In [5]:
df.columns

Index(['key', 'datasetKey', 'publishingOrgKey', 'installationKey',
       'hostingOrganizationKey', 'publishingCountry', 'protocol',
       'lastCrawled', 'lastParsed', 'crawlId',
       ...
       'higherClassification', 'georeferenceSources', 'typeStatus',
       'georeferencedDate', 'georeferenceProtocol', 'locationAccordingTo',
       'footprintWKT', 'verbatimCoordinateSystem', 'organismID',
       'previousIdentifications'],
      dtype='object', length=140)

In [6]:
lista = df.columns

In [7]:
lista

Index(['key', 'datasetKey', 'publishingOrgKey', 'installationKey',
       'hostingOrganizationKey', 'publishingCountry', 'protocol',
       'lastCrawled', 'lastParsed', 'crawlId',
       ...
       'higherClassification', 'georeferenceSources', 'typeStatus',
       'georeferencedDate', 'georeferenceProtocol', 'locationAccordingTo',
       'footprintWKT', 'verbatimCoordinateSystem', 'organismID',
       'previousIdentifications'],
      dtype='object', length=140)

In [8]:
for e in lista:
    print(e)

key
datasetKey
publishingOrgKey
installationKey
hostingOrganizationKey
publishingCountry
protocol
lastCrawled
lastParsed
crawlId
extensions
basisOfRecord
occurrenceStatus
taxonKey
kingdomKey
phylumKey
classKey
orderKey
familyKey
genusKey
speciesKey
acceptedTaxonKey
scientificName
acceptedScientificName
kingdom
phylum
order
family
genus
species
genericName
specificEpithet
taxonRank
taxonomicStatus
iucnRedListCategory
dateIdentified
decimalLatitude
decimalLongitude
continent
stateProvince
gadm
year
month
day
eventDate
startDayOfYear
endDayOfYear
issues
modified
lastInterpreted
references
license
isSequenced
identifiers
media
facts
relations
isInCluster
datasetName
recordedBy
identifiedBy
geodeticDatum
class
countryCode
recordedByIDs
identifiedByIDs
gbifRegion
country
publishedByGbifRegion
rightsHolder
identifier
http://unknown.org/nick
verbatimEventDate
collectionCode
verbatimLocality
gbifID
occurrenceID
taxonID
catalogNumber
institutionCode
eventTime
http://unknown.org/captive
identific

In [9]:
colunas_relevantes = ["key","publishingCountry", "protocol", "basisOfRecord", "scientificName", "acceptedScientificName", "kingdom",
"phylum", "order", "family", "genus", "species", "genericName", "specificEpithet", "taxonRank", "taxonomicStatus", "iucnRedListCategory",
"dateIdentified", "decimalLatitude", "decimalLongitude", "continent", "stateProvince", "recordedBy", "identifiedBy", "class", 
"rightsHolder", "coordinateUncertaintyInMeters"]
df = df[colunas_relevantes]
df.head()

,key,publishingCountry,protocol,basisOfRecord,scientificName,acceptedScientificName,kingdom,phylum,order,family,genus,species,genericName,specificEpithet,taxonRank,taxonomicStatus,iucnRedListCategory,dateIdentified,decimalLatitude,decimalLongitude,continent,stateProvince,recordedBy,identifiedBy,class,rightsHolder,coordinateUncertaintyInMeters
0,4854782149,US,DWC_ARCHIVE,HUMAN_OBSERVATION,Hippobroma longiflora (L.) G.Don,Hippobroma longiflora (L.) G.Don,Plantae,Tracheophyta,Asterales,Campanulaceae,Hippobroma,Hippobroma longiflora,Hippobroma,longiflora,SPECIES,ACCEPTED,NE,2024-05-04T01:48:20,-1.504886,-48.462257,SOUTH_AMERICA,Pará,Fernando J. M. Rojas-Runjaic,Fernando J. M. Rojas-Runjaic,Magnoliopsida,Fernando J. M. Rojas-Runjaic,NaN
1,4868048393,US,DWC_ARCHIVE,HUMAN_OBSERVATION,Bonnaya antipoda (L.) Druce,Bonnaya antipoda (L.) Druce,Plantae,Tracheophyta,Lamiales,Linderniaceae,Bonnaya,Bonnaya antipoda,Bonnaya,antipoda,SPECIES,ACCEPTED,LC,2024-05-04T01:48:20,-1.504778,-48.462644,SOUTH_AMERICA,Pará,Fernando J. M. Rojas-Runjaic,Fernando J. M. Rojas-Runjaic,Magnoliopsida,Fernando J. M. Rojas-Runjaic,NaN
2,4855043797,US,DWC_ARCHIVE,HUMAN_OBSERVATION,"Iguana iguana (Linnaeus, 1758)","Iguana iguana (Linnaeus, 1758)",Animalia,Chordata,NaN,Iguanidae,Iguana,Iguana iguana,Iguana,iguana,SPECIES,ACCEPTED,LC,2024-05-04T01:48:22,-1.505205,-48.462697,SOUTH_AMERICA,Pará,Fernando J. M. Rojas-Runjaic,Fernando J. M. Rojas-Runjaic,Squamata,Fernando J. M. Rojas-Runjaic,NaN
3,4891949736,US,DWC_ARCHIVE,HUMAN_OBSERVATION,"Avicularia avicularia (Linnaeus, 1758)","Avicularia avicularia (Linnaeus, 1758)",Animalia,Arthropoda,Araneae,Theraphosidae,Avicularia,Avicularia avicularia,Avicularia,avicularia,SPECIES,ACCEPTED,NE,2024-06-08T14:22:25,-1.511736,-48.483067,SOUTH_AMERICA,Pará,globalherping,Jean Martins,Arachnida,globalherping,31446.0
4,4872372054,US,DWC_ARCHIVE,HUMAN_OBSERVATION,"Butorides striata (Linnaeus, 1758)","Butorides striata (Linnaeus, 1758)",Animalia,Chordata,Pelecaniformes,Ardeidae,Butorides,Butorides striata,Butorides,striata,SPECIES,ACCEPTED,LC,2024-05-27T14:42:31,-1.493230,-48.461513,SOUTH_AMERICA,Pará,Gustavo Melo,Gustavo Melo,Aves,Gustavo Melo,61.0


In [10]:
df.columns

Index(['key', 'publishingCountry', 'protocol', 'basisOfRecord',
       'scientificName', 'acceptedScientificName', 'kingdom', 'phylum',
       'order', 'family', 'genus', 'species', 'genericName', 'specificEpithet',
       'taxonRank', 'taxonomicStatus', 'iucnRedListCategory', 'dateIdentified',
       'decimalLatitude', 'decimalLongitude', 'continent', 'stateProvince',
       'recordedBy', 'identifiedBy', 'class', 'rightsHolder',
       'coordinateUncertaintyInMeters'],
      dtype='object')

In [11]:
df.shape

(610, 27)